In [6]:
#encoding=utf-8
import argparse
from utils import *
from torch.utils.data.dataset import *
from torch.utils.data.sampler import *  #用于采样
from torch.nn.utils.rnn import *
import bisect
from model import *
import torch
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time



In [7]:
parser = argparse.ArgumentParser(description="Joint Extraction of Entities and Relations")
parser.add_argument('--batch_size', type=int, default=32, metavar='N',
                    help='batch size (default: 32)')
parser.add_argument('--cuda', action='store_false',default=False,
                    help='use CUDA (default: True)')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='dropout applied to layers (default: 0.5)')
parser.add_argument('--emb_dropout', type=float, default=0.25,
                    help='dropout applied to the embedded layer (default: 0.25)')
parser.add_argument('--clip', type=float, default=0.35,
                    help='gradient clip, -1 means no clip (default: 0.35)')
parser.add_argument('--epochs', type=int, default=30,
                    help='upper epoch limit (default: 30)')
parser.add_argument('--char_kernel_size', type=int, default=3,
                    help='character-level kernel size (default: 3)')
parser.add_argument('--word_kernel_size', type=int, default=3,
                    help='word-level kernel size (default: 3)')
parser.add_argument('--emsize', type=int, default=50,
                    help='size of character embeddings (default: 50)')
parser.add_argument('--char_layers', type=int, default=3,
                    help='# of character-level convolution layers (default: 3)')
parser.add_argument('--word_layers', type=int, default=3,
                    help='# of word-level convolution layers (default: 3)')
parser.add_argument('--char_nhid', type=int, default=50,
                    help='number of hidden units per character-level convolution layer (default: 50)')
parser.add_argument('--word_nhid', type=int, default=300,
                    help='number of hidden units per word-level convolution layer (default: 300)')
parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                    help='report interval (default: 100)')
parser.add_argument('--lr', type=float, default=4,
                    help='initial learning rate (default: 4)')
parser.add_argument('--optim', type=str, default='SGD',
                    help='optimizer type (default: SGD)')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed (default: 1111)')
parser.add_argument('--save', type=str, default='model.pt',
                    help='path to save the final model')
parser.add_argument('--weight', type=float, default=10.0,
                    help='manual rescaling weight given to each tag except "O"')
args = parser.parse_args()


usage: ipykernel_launcher.py [-h] [--batch_size N] [--cuda]
                             [--dropout DROPOUT] [--emb_dropout EMB_DROPOUT]
                             [--clip CLIP] [--epochs EPOCHS]
                             [--char_kernel_size CHAR_KERNEL_SIZE]
                             [--word_kernel_size WORD_KERNEL_SIZE]
                             [--emsize EMSIZE] [--char_layers CHAR_LAYERS]
                             [--word_layers WORD_LAYERS]
                             [--char_nhid CHAR_NHID] [--word_nhid WORD_NHID]
                             [--log-interval N] [--lr LR] [--optim OPTIM]
                             [--seed SEED] [--save SAVE] [--weight WEIGHT]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\pc\AppData\Roaming\jupyter\runtime\kernel-677ce300-c4b7-4e3e-b51a-d8cd9e39c393.json


SystemExit: 2

c:\program files\python\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
# Set the random seed manually for reproducibility.
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

print(args)
device = torch.device("cuda" if args.cuda else "cpu")#torch.device代表将torch.Tensor分配到的设备的对象



NameError: name 'args' is not defined

In [ ]:

charset = Charset()#定义字符对象
vocab = Vocabulary()#定义词汇对象
vocab.load("data/myvocab.txt")#读文件
tag_set = Index()#定义对象
tag_set.load("data/my_tag2id.txt")#读文件  标记-id
relation_labels = Index()#定义对象
relation_labels.load('data/my_relation_labels.txt')#读文件   关系标签



In [ ]:
##这里导入的是处理过后的训练集和测试集.pk
train_data = load('data/my_train.pk')
#test_data = load('data/NYT_CoType/test.pk')

#print(len(train_data[:500]))
#print(type(train_data))
#print(len(train_data),len(val_data),len(test_data))
#train_data=train_data[:1000]
#test_data=test_data[:100]
val_size = int(0.02 * len(train_data))#设置验证集大小
train_data, val_data = random_split(train_data, [len(train_data)-val_size, val_size])##torch里的函数 随机切分训练集和验证集




In [ ]:
def group(data, breakpoints):# 把数据集按照句子长度分组#data:包含三部分 句子中每个词的id 句子中每个词对应的char的id 句子对应的实体id
    groups = [[] for _ in range(len(breakpoints)+1)]
    for idx, item in enumerate(data):
        i = bisect.bisect_left(breakpoints, len(item[0]))
        groups[i].append(idx)
    data_groups = [Subset(data, g) for g in groups]  #Subset是torch.utils.data.dataset中的
    return data_groups


In [ ]:
#三个集合都按照句子的长度分了组
train_data_groups = group(train_data, [60, 80, 100, 120, 140, 160])
val_data_groups = group(val_data, [60, 80, 100, 120, 140, 160])
#test_data_groups = group(test_data, [10, 20, 30, 40, 50, 60])
#print(len(train_data_groups))



In [ ]:
class GroupBatchRandomSampler(object):##每一组的句子长度不同 分别建采样器进行随机采样
    def __init__(self, data_groups, batch_size, drop_last):
        self.batch_indices = []
        for data_group in data_groups:
            self.batch_indices.extend(list(BatchSampler(SubsetRandomSampler(data_group.indices),
                                                        batch_size, drop_last=drop_last)))

    def __iter__(self):
        return (self.batch_indices[i] for i in torch.randperm(len(self.batch_indices)))

    def __len__(self):
        return len(self.batch_indices)



In [ ]:
def get_batch(batch_indices, data):  #分batch   data有三部分   返回三部分的batch
    #print(batch_indices,len(data))
    batch = [data[idx] for idx in batch_indices]
    sorted_batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)
    sentences, tokens, tags = zip(*sorted_batch)

    padded_sentences, lengths = pad_packed_sequence(pack_sequence([torch.LongTensor(_) for _ in sentences]),  #文本长度对齐
                                                    batch_first=True, padding_value=vocab["<pad>"])
    padded_tokens, _ = pad_packed_sequence(pack_sequence([torch.LongTensor(_) for _ in tokens]),
                                           batch_first=True, padding_value=charset["<pad>"])
    padded_tags, _ = pad_packed_sequence(pack_sequence([torch.LongTensor(_) for _ in tags]),
                                         batch_first=True, padding_value=tag_set["O"])

    return padded_sentences.to(device), padded_tokens.to(device), padded_tags.to(device), lengths.to(device)



In [ ]:
word_embeddings = torch.tensor(np.load("data/NYT_CoType/word2vec.vectors.npy"))      #将词向量转换为张量 47463*300
word_embedding_size = word_embeddings.size(1)
pad_embedding = torch.empty(1, word_embedding_size).uniform_(-0.5, 0.5)  #随机初始化 1*word_embedding_size 大小的张量 取值在-0.5~0.5之间
unk_embedding = torch.empty(1, word_embedding_size).uniform_(-0.5, 0.5)
word_embeddings = torch.cat([pad_embedding, unk_embedding, word_embeddings])


In [ ]:
###定义两个模型的层数和大小
char_channels = [args.emsize] + [args.char_nhid] * args.char_layers
word_channels = [word_embedding_size + args.char_nhid] + [args.word_nhid] * args.word_layers


In [ ]:
if os.path.exists("model.pt"):      #是否存在已经构造好的模型
    model=torch.load('model.pt')
#——————————搭建模型————————————
else:
    model = Model(charset_size=len(charset), char_embedding_size=args.emsize, char_channels=char_channels,
                  char_padding_idx=charset["<pad>"], char_kernel_size=args.char_kernel_size, weight=word_embeddings,
                  word_embedding_size=word_embedding_size, word_channels=word_channels,
                  word_kernel_size=args.word_kernel_size, num_tag=len(tag_set), dropout=args.dropout,
                  emb_dropout=args.emb_dropout).to(device)




In [ ]:
weight = [args.weight] * len(tag_set)   #生成一维向量 初始化值为arg.weight 维度为len(tag_set)
weight[tag_set["O"]] = 1     #"O"标签对应的权值为1
weight = torch.tensor(weight).to(device)   #将weight转换为张量形式 device:分配到设备 这里是cpu
criterion = nn.NLLLoss(weight, size_average=False)      #定义的损失函数
optimizer = getattr(optim, args.optim)(model.parameters(), lr=args.lr)  #getattr 获取torch.optim对象的属性值 默认是SGD(随机梯度下降)   得到优化器的损失函数(模型参数)



In [ ]:
def train():
    model.train()    #torch的模型训练
    total_loss = 0
    count = 0
    sampler = GroupBatchRandomSampler(train_data_groups, args.batch_size, drop_last=False)#采样器是定义的采样器对象 按组分batch分别采样

    for idx, batch_indices in enumerate(sampler):
        sentences, tokens, targets, lengths = get_batch(batch_indices, train_data) #分batch
        optimizer.zero_grad()
        output = model(sentences, tokens)  #词级别和字符级别建模
        output = pack_padded_sequence(output, lengths, batch_first=True).data #文本长度对齐
        targets = pack_padded_sequence(targets, lengths, batch_first=True).data #将标签的编码长度对齐
        loss = criterion(output, targets)  #损失函数
        loss.backward()
        if args.clip > 0:
            nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()

        total_loss += loss.item()
        count += len(targets)
        if (idx+1) % args.log_interval == 0:
            cur_loss = total_loss / count
            elapsed = time.time() - start_time
            percent = ((epoch-1)*len(sampler)+(idx+1))/(args.epochs*len(sampler))
            remaining = elapsed / percent - elapsed
            print("| Epoch {:2d}/{:2d} | Batch {:5d}/{:5d} | Elapsed Time {:s} | Remaining Time {:s} | "
                  "lr {:4.2e} | Loss {:5.3f} |".format(epoch, args.epochs, idx+1, len(sampler), time_display(elapsed),
                                                       time_display(remaining), lr, cur_loss))
            total_loss = 0
            count = 0




In [ ]:
def evaluate(data_groups):
    model.eval()
    total_loss = 0
    count = 0.000000001
    TP = 0.000000001
    TP_FP = 0.000000001
    TP_FN = 0.000000001
    with torch.no_grad():
        for batch_indices in GroupBatchRandomSampler(data_groups, args.batch_size, drop_last=False):
            sentences, tokens, targets, lengths = get_batch(batch_indices, train_data)
            output = model(sentences, tokens)
            tp, tp_fp, tp_fn = measure(output, targets, lengths)
            TP += tp
            TP_FP += tp_fp
            TP_FN += tp_fn
            output = pack_padded_sequence(output, lengths, batch_first=True).data
            targets = pack_padded_sequence(targets, lengths, batch_first=True).data
            loss = criterion(output, targets)
            total_loss += loss.item()
            count += len(targets)
            print(count,TP_FP,TP_FN)
    return total_loss / count, TP/TP_FP, TP/TP_FN, 2*TP/(TP_FP+TP_FN)



In [ ]:
def measure(output, targets, lengths):
    assert output.size(0) == targets.size(0) and targets.size(0) == lengths.size(0)
    tp = 0
    tp_fp = 0
    tp_fn = 0
    batch_size = output.size(0)
    output = torch.argmax(output, dim=-1)
    for i in range(batch_size):
        length = lengths[i]
        out = output[i][:length].tolist()
        target = targets[i][:length].tolist()
        out_triplets = get_triplets(out)
        print("out:",out_triplets)
        tp_fp += len(out_triplets)
        target_triplets = get_triplets(target)
        print("target:",target_triplets)
        tp_fn += len(target_triplets)
        for target_triplet in target_triplets:
            for out_triplet in out_triplets:
                if out_triplet == target_triplet:
                    tp += 1
    return tp, tp_fp, tp_fn



In [ ]:
def get_triplets(tags):
    temp = {}
    triplets = []
    for idx, tag in enumerate(tags):
        if tag == tag_set["O"]:
            continue
        pos, relation_label, role = tag_set[tag].split("-")
        if pos == "B" or pos == "S":
            if relation_label not in temp:
                temp[relation_label] = [[], []]
            temp[relation_label][int(role) - 1].append(idx)
    for relation_label in temp:
        role1, role2 = temp[relation_label]
        if role1 and role2:
            len1, len2 = len(role1), len(role2)
            if len1 > len2:
                for e2 in role2:
                    idx = np.argmin([abs(e2 - e1) for e1 in role1])
                    e1 = role1[idx]
                    triplets.append((e1, relation_label, e2))
                    del role1[idx]
            else:
                for e1 in role1:
                    idx = np.argmin([abs(e2 - e1) for e2 in role2])
                    e2 = role2[idx]
                    triplets.append((e1, relation_label, e2))
                    del role2[idx]
    return triplets



In [ ]:
if __name__ == "__main__":
    best_val_loss = None
    lr = args.lr
    all_val_loss = []
    all_precision = []
    all_recall = []
    all_f1 = []

    # At any point you can hit Ctrl + C to break out of training early.
    try:
        start_time = time.time()
        print("-" * 118)
        for epoch in range(1, args.epochs+1):
            train()
            val_loss, precision, recall, f1 = evaluate(val_data_groups)

            elapsed = time.time() - start_time
            print("-" * 118)
            print("| End of Epoch {:2d} | Elapsed Time {:s} | Validation Loss {:5.3f} | Precision {:5.3f} "
                  "| Recall {:5.3f} | F1 {:5.3f} |".format(epoch, time_display(elapsed),
                                                           val_loss, precision, recall, f1))
            print("-" * 118)

            # Save the model if the validation loss is the best we've seen so far.
            if not best_val_loss or val_loss < best_val_loss:
                with open(args.save, 'wb') as f:
                    torch.save(model, f)
                best_val_loss = val_loss
            else:
                # Anneal the learning rate if no improvement has been seen in the validation dataset.
                lr = lr / 4.0
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
            all_val_loss.append(val_loss)
            all_precision.append(precision)
            all_recall.append(recall)
            all_f1.append(f1)

    except KeyboardInterrupt:
        print('-' * 118)
        print('Exiting from training early')

    # Load the best saved model.
    with open(args.save, 'rb') as f:
        model = torch.load(f)

    # Run on test data
    test_loss, precision, recall, f1 = evaluate(test_data_groups)
    print("=" * 118)
    print("| End of Training | Test Loss {:5.3f} | Precision {:5.3f} "
          "| Recall {:5.3f} | F1 {:5.3f} |".format(test_loss, precision, recall, f1))
    print("=" * 118)

    with open("record.tsv", "wt", encoding="utf-8") as f:
        for idx in range(len(all_val_loss)):
            f.write("{:d}\t{:5.3f}\t{:5.3f}\t{:5.3f}\t{:5.3f}\n"
                    .format(idx+1, all_val_loss[idx], all_precision[idx], all_recall[idx], all_f1[idx]))
        f.write("\n{:5.3f}\t{:5.3f}\t{:5.3f}\t{:5.3f}\n".format(test_loss, precision, recall, f1))
